Create streaming dataframe that reads data from source increamently.

Setting up streaming query that uses autoloader to moniter a cloud storage for new files

In [0]:
checkpoint_location= "abfss://checkpoints@storageformetastore.dfs.core.windows.net/"

df= spark.readStream.format("cloudFiles").option("cloudFiles.format","csv")\
    .option("cloudFiles.schemaLocation",checkpoint_location)\
    .load("abfss://raw@storageforproject.dfs.core.windows.net/")
        

Write stream method allows you to continuously append data to the target starage as new files arrive.

Checkpoint ensures that the write operations is fault-tolerant, if the stream pipeline fails. It can resume from where it left off without duplicate data.

In [0]:
df.writeStream\
    .format("csv")\
    .outputMode("append")\
    .option("header", "true")\
    .option("checkpointLocation", checkpoint_location)\
    .trigger(processingTime="10 seconds")\
    .start("abfss://bronze@storageforproject.dfs.core.windows.net/netflix_titles/")

In [0]:
df_read = spark.read.format("csv")\
    .option("header", "true")\
    .load("abfss://bronze@storageforproject.dfs.core.windows.net/netflix_titles/")

display(df_read)

`cloudFiles` is used because Auto Loader simplifies the ingestion process, especially for continuously arriving data. It automatically detects new files, tracks processed ones, and handles schema evolution. If we used `readStream` without specifying `cloudFiles`, we would have to handle these aspects manually, which is less efficient and more error-prone.